In [1]:
from dataclasses import dataclass
from typing import List, Tuple, Dict


@dataclass(frozen=True)
class ResultCell:
    i: int
    k: int


@dataclass
class Val:
    tag: str
    j: int
    value: float


def mapper_a(A: List[List[float]], cols_b: int) -> List[Tuple[ResultCell, Val]]:
    rows = len(A)
    cols = len(A[0])
    result = []
    for i in range(rows):
        for j in range(cols):
            cell_value = A[i][j]
            for k in range(cols_b):
                result.append((ResultCell(i, k), Val('A', j, cell_value)))
    return result


def mapper_b(B: List[List[float]], rows_a: int) -> List[Tuple[ResultCell, Val]]:
    rows = len(B)
    cols = len(B[0])
    result = []
    for j in range(rows):
        for k in range(cols):
            cell_value = B[j][k]
            for i in range(rows_a):
                result.append((ResultCell(i, k), Val('B', j, cell_value)))
    return result


def shuffler(mapped: List[Tuple[ResultCell, Val]]) -> Dict[ResultCell, List[Val]]:
    sorted_mapped = sorted(mapped, key=lambda x: (x[0].i, x[0].k, x[1].tag, x[1].j))
    result: Dict[ResultCell, List[Val]] = {}
    for key, value in sorted_mapped:
        if key not in result:
            result[key] = []
        result[key].append(value)
    return result


def reducer(grouped: Dict[ResultCell, List[Val]], rows_c: int, cols_c: int) -> List[List[float]]:
    C = [[0.0] * cols_c for _ in range(rows_c)]
    
    for result_cell, values in grouped.items():
        row_values_a: Dict[int, float] = {}
        column_values_b: Dict[int, float] = {}
        
        for val in values:
            if val.tag == 'A':
                row_values_a[val.j] = val.value
            elif val.tag == 'B':
                column_values_b[val.j] = val.value
        
        total = 0.0
        for j, value_a in row_values_a.items():
            if j in column_values_b:
                total += value_a * column_values_b[j]
        
        C[result_cell.i][result_cell.k] = total
    
    return C


def multiply_map_reduce(A: List[List[float]], B: List[List[float]]) -> List[List[float]]:

    rows_a = len(A)
    cols_a = len(A[0])
    rows_b = len(B)
    cols_b = len(B[0])
    
    if cols_a != rows_b:
        raise ValueError(
            f"Размерности матриц не позволяют выполнить операцию умножения:\n\tA - {rows_a}x{cols_a}, B - {rows_b}x{cols_b}"
        )
    
    mapped_a = mapper_a(A, cols_b)
    mapped_b = mapper_b(B, rows_a)
    
    grouped = shuffler(mapped_a + mapped_b)
    
    return reducer(grouped, rows_a, cols_b)

if __name__ == "__main__":
    A = [
        [3,5,7],
        [1,2,3]
    ]
    B = [
        [1,2],
        [3,4],
        [5,6]
    ]
    
    C = multiply_map_reduce(A, B)
    
    for row in C:
        print(", ".join(f"{x}" for x in row))

53.0, 68.0
22.0, 28.0
